### Init

In [ ]:
import ipywidgets as widgets
from ipywidgets import  Layout
from IPython.display import display,Javascript
import pandas as pd
import numpy as np
hitDf=pd.read_pickle('./tmp.profile.pickle')
inputAnalyzeDf=np.log2(hitDf+1)
listOfQueries=list(inputAnalyzeDf.columns.get_level_values('queryLabel').unique())
wA=widgets.Dropdown(
    options=listOfQueries,value=listOfQueries[0],description='condition A:',disabled=False,
)
wB=widgets.Dropdown(
    options=listOfQueries,    value=listOfQueries[1],description='condition B:',disabled=False,
)
def run_all(ev):
    display(Javascript('IPython.notebook.execute_cells_below()'))
button_query = widgets.Button(description="Generate plot",
                             layout=Layout(width='20%', height='100%'))
button_query.on_click(run_all)
button_query.style.button_color='lightblue'

### SELECT THE COMPARISON CLASS HERE AND CLICK BLUE BUTTON "Generate plot"

In [ ]:
display(wA)
display(wB)
display(button_query)

### Code

In [ ]:
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode()

In [ ]:
from scipy import stats
expression_effect_size_filter=0.0 
labelA=wA.get_interact_value()
labelB=wB.get_interact_value()
t,p=stats.ttest_ind(inputAnalyzeDf[labelA],inputAnalyzeDf[labelB],axis=1)
effectDiff=inputAnalyzeDf[labelA].mean(axis=1)-inputAnalyzeDf[labelB].mean(axis=1)
effectLabel='expression of : "{}" - "{}"'.format(labelA, labelB)
tmpDf=pd.DataFrame({'t':t,'-log10(p)':-np.log10(p),effectLabel:effectDiff,'u':inputAnalyzeDf.mean(axis=1)},index=inputAnalyzeDf.index)
plotDf=tmpDf[tmpDf['u']>=expression_effect_size_filter].dropna()
yLabel='-log10(p)'
xLabel=effectLabel

layout = go.Layout(
    yaxis={'title':"-log10(p)"},
    xaxis={'title':"{} - {}".format(labelA,labelB)}
)
fig = go.Figure(layout=layout)
#m=plotDf.index.isin(annotateGenesList)
fig.add_scatter( x=plotDf[xLabel],y=plotDf[yLabel],mode='markers',hovertext=plotDf.index.values,name='')



### Plot

In [ ]:
iplot(fig)